### Part 3. Langchain - VectorDB 이용한 간단한 RAG 구현
- Objectives: Langchain+파인콘으로 간단한 위키문서 upsert 및 이를 기반으로 하는 naive-RAG 구현

In [1]:
import os
import time
from dotenv import load_dotenv
load_dotenv()

pinecone_api_key = os.environ.get('PINECONE_API_KEY')
openai_api_key = os.environ.get('OPENAI_API_KEY')

In [2]:
#https://huggingface.co/datasets/lcw99/wikipedia-korean-20221001/viewer/default/train
# 사용 데이터 로드
from datasets import load_dataset

dataset = load_dataset("lcw99/wikipedia-korean-20221001", split='train[:100]') # 데이터 전부 사용시 너무 방대, 따라서 100개만 사용

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/607256 [00:00<?, ? examples/s]

In [5]:
# dataset[0]

In [4]:
# 텍스트 스플리터 기능을 활용한 데이터 문서 청킹 작업 정의
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=400,
    chunk_overlap=20,
    separators=["\n\n", "\n", " ", ""]
)

In [6]:
# 정의한 청킹 방식으로 어떤 식으로 짤라지는지 확인.
text_splitter.split_text(dataset[0]['text'])[:5]

['제임스 얼 카터 주니어(, 1924년 10월 1일 ~ )는 민주당 출신 미국 39대 대통령 (1977년 ~ 1981년)이다.\n\n생애\n\n어린 시절 \n지미 카터는 조지아주 섬터 카운티 플레인스 마을에서 태어났다.\n\n조지아 공과대학교를 졸업하였다. 그 후 해군에 들어가 전함·원자력·잠수함의 승무원으로 일하였다. 1953년 미국 해군 대위로 예편하였고 이후 땅콩·면화 등을 가꿔 많은 돈을 벌었다. 그의 별명이 "땅콩 농부" (Peanut Farmer)로 알려졌다.',
 '정계 입문 \n1962년 조지아주 상원 의원 선거에서 낙선하나 그 선거가 부정선거 였음을 입증하게 되어 당선되고, 1966년 조지아 주지사 선거에 낙선하지만, 1970년 조지아 주지사를 역임했다. 대통령이 되기 전 조지아주 상원의원을 두번 연임했으며, 1971년부터 1975년까지 조지아 지사로 근무했다. 조지아 주지사로 지내면서, 미국에 사는 흑인 등용법을 내세웠다.\n\n대통령 재임 \n\n1976년 미합중국 제39대 대통령 선거에 민주당 후보로 출마하여 도덕주의 정책으로 내세워서, 많은 지지를 받고 제럴드 포드 대통령을 누르고 당선되었다.\n\n카터 대통령은 에너지 개발을 촉구했으나 공화당의 반대로 무산되었다.',
 '외교 정책 \n카터는 이집트와 이스라엘을 조정하여 캠프 데이비드에서 안와르 사다트 대통령과 메나헴 베긴 수상과 함께 중동 평화를 위한 캠프데이비드 협정을 체결했다. 이것은 공화당과 미국의 유대인 단체의 반발을 일으켰다. 그러나 1979년, 양국 간의 평화조약이 백악관에서 이루어졌다.\n\n소련과 제2차 전략 무기 제한 협상(SALT II)에 조인했다.\n\n카터는 1970년대 후반 당시 대한민국 등 인권 후진국의 국민들의 인권을 지키기 위해 노력했으며, 취임 이후 계속해서 도덕정치를 내세웠다.\n\n임기 말, 소련의 아프가니스탄 침공 사건으로 인해 1980년 하계 올림픽에 반공국가들의 보이콧을 하였다.',
 '그는 주이란 미국 대사관 인질 사건의 인질 구출 실패로 인

In [8]:
# 청크 단위 임베딩이 필요하므로 임베딩 벡터 불러오기
from langchain_openai import OpenAIEmbeddings

model_name = 'text-embedding-ada-002'

embed = OpenAIEmbeddings(
    model=model_name,
    openai_api_key=openai_api_key
)

In [9]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key=pinecone_api_key)

In [10]:
from time import sleep

index_name="quickstart"

if index_name in [index_info["name"] for index_info in pc.list_indexes()]:
    pc.delete_index(index_name)

pc.create_index(
   name=index_name,
   dimension=1536,
   metric="dotproduct",
   spec=ServerlessSpec(
       cloud='aws',
       region='us-east-1'
   )
)

while not pc.describe_index(index_name).status['ready']:
    sleep(1)
index = pc.Index(index_name)
sleep(1)
index_stats = index.describe_index_stats()
print(index_stats)

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}


In [11]:
from tqdm.auto import tqdm
from uuid import uuid4

def process_and_upload_records(dataset, batch_limit=100):
    texts = []
    metadatas = []

    for i, record in enumerate(tqdm(dataset)):
        metadata = {
            'id': str(record['id']),
            'source': record['url'],
            'title': record['title']
        }

        record_texts = text_splitter.split_text(record['text'])
        record_metadatas = [
            {"chunk": j, "text": text, **metadata}
            for j, text in enumerate(record_texts)
        ]

        texts.extend(record_texts)
        metadatas.extend(record_metadatas)

        if len(texts) >= batch_limit:
            upload_data(texts, metadatas)
            texts, metadatas = [], []

    if texts:
        upload_data(texts, metadatas)

def upload_data(texts, metadatas):
    # chunk 단위 embedding
    ids = [str(uuid4()) for _ in range(len(texts))]
    embeds = embed.embed_documents(texts)
    index.upsert(vectors=zip(ids, embeds, metadatas))

process_and_upload_records(dataset)

  0%|          | 0/100 [00:00<?, ?it/s]

In [12]:
# upsert 후 확인
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 1521}},
 'total_vector_count': 1521}

In [13]:
#파인콘 api로 기반 문서들 업서트 이후에는 이제 랭체인에서 이를 활용하기 위해 벡터스토어 오브젝트로 연결시켜줌
from langchain.vectorstores import Pinecone

text_field = "text"

vectorstore = Pinecone(
    index, embed.embed_query, text_field
)

C:\Users\sckim\.conda\envs\vectordb\Lib\site-packages\langchain_community\vectorstores\pinecone.py:75: UserWarning: Passing in `embedding` as a Callable is deprecated. Please pass in an Embeddings object instead.
  warnings.warn(


In [14]:
# 단순 retrieval
query = "지미 카터가 누구야?"

vectorstore.similarity_search(
    query,
    k=3
)

[Document(page_content='제임스 얼 카터 주니어(, 1924년 10월 1일 ~ )는 민주당 출신 미국 39대 대통령 (1977년 ~ 1981년)이다.\n\n생애\n\n어린 시절 \n지미 카터는 조지아주 섬터 카운티 플레인스 마을에서 태어났다.\n\n조지아 공과대학교를 졸업하였다. 그 후 해군에 들어가 전함·원자력·잠수함의 승무원으로 일하였다. 1953년 미국 해군 대위로 예편하였고 이후 땅콩·면화 등을 가꿔 많은 돈을 벌었다. 그의 별명이 "땅콩 농부" (Peanut Farmer)로 알려졌다.', metadata={'chunk': 0.0, 'id': '5', 'source': 'https://ko.wikipedia.org/wiki/%EC%A7%80%EB%AF%B8%20%EC%B9%B4%ED%84%B0', 'title': '지미 카터'}),
 Document(page_content='퇴임 이후 \n\n퇴임 이후 민간 자원을 적극 활용한 비영리 기구인 카터 재단을 설립한 뒤 민주주의 실현을 위해 제 3세계의 선거 감시 활동 및 기니 벌레에 의한 드라쿤쿠르스 질병 방재를 위해 힘썼다. 미국의 빈곤층 지원 활동, 사랑의 집짓기 운동, 국제 분쟁 중재 등의 활동도 했다.\n\n카터는 카터 행정부 이후 미국이 북핵 위기, 코소보 전쟁, 이라크 전쟁과 같이 미국이 군사적 행동을 최후로 선택하는 전통적 사고를 버리고 군사적 행동을 선행하는 행위에 대해 깊은 유감을 표시 하며 미국의 군사적 활동에 강한 반대 입장을 보이고 있다.', metadata={'chunk': 6.0, 'id': '5', 'source': 'https://ko.wikipedia.org/wiki/%EC%A7%80%EB%AF%B8%20%EC%B9%B4%ED%84%B0', 'title': '지미 카터'}),
 Document(page_content='지미 카터는 대한민국과의 관계에서도 중요한 영향을 미쳤던 대통령 중 하나다. 인권 문제와 주한미군 철수 문제로 한때 한미 관계가 불편하기도 했다

In [15]:
# full RAG 구성
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA

llm = ChatOpenAI(
    openai_api_key=openai_api_key,
    model_name='gpt-3.5-turbo',
    temperature=0
)

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever()
)

C:\Users\sckim\.conda\envs\vectordb\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [17]:
# 사용자 질문 -> similarity Search -> GPT 답안 생성
qa.invoke(query)

{'query': '지미 카터가 누구야?',
 'result': '제임스 얼 카터 주니어는 민주당 출신 미국 39대 대통령으로, 1977년부터 1981년까지 대통령을 역임한 인물입니다. 그는 땅콩 농부로도 알려져 있습니다. 퇴임 이후에는 카터 재단을 설립하여 민주주의 실현을 위해 활동하고 있습니다. 또한 대한민국과의 관계에서도 중요한 영향을 미쳤던 대통령 중 한 명입니다.'}

In [18]:
# 사용된 소스를 확인할 수 있는 소스체인 구성
from langchain.chains import RetrievalQAWithSourcesChain

qa_with_sources = RetrievalQAWithSourcesChain.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever()
)

In [19]:
# 소스체인 RAG 시험
qa_with_sources(query)

C:\Users\sckim\.conda\envs\vectordb\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


{'question': '지미 카터가 누구야?',
 'answer': '지미 카터는 미국 39대 대통령이다.\n',
 'sources': 'https://ko.wikipedia.org/wiki/%EC%A7%80%EB%AF%B8%20%EC%B9%B4%ED%84%B0'}